# LCU Select/Prepare Oracles

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `LCUBlockEncoding`
LCU based block encoding using SELECT and PREPARE oracles.

Builds the block encoding via
$$
    B[H] = \mathrm{SELECT}
$$

$$
    \mathrm{SELECT} |l\rangle_a|\psi\rangle_s = |l\rangle_a U_l |\psi\rangle_s.
$$

The Hamiltonian can be extracted via

$$
    \langle G | B[H] | G \rangle = H / \alpha
$$

where

$$
    |G\rangle = \mathrm{PREPARE} |0\rangle_a = \sum_l \sqrt{\frac{w_l}{\alpha}} |l\rangle_a,
$$

The ancilla register is at least of size $\log L$.

In our implementations we typically split the ancilla registers into
selection registers (i.e.  the $l$ registers above) and junk registers which
are extra qubits needed by state preparation but not controlled upon during
SELECT.

#### Parameters
 - `alpha`: The normalization constant upper bounding the spectral norm of the Hamiltonian. Often called lambda.
 - `epsilon`: The precision to which the block encoding is performed. Currently this isn't used: see https://github.com/quantumlib/Qualtran/issues/985
 - `select`: The bloq implementing the `SelectOracle` interface.
 - `prepare`: The bloq implementing the `PrepareOracle` interface. 

#### Registers
 - `selection`: The combined selection register.
 - `junk`: Additional junk registers not prepared upon.
 - `system`: The combined system register. 

#### References
 - [Hamiltonian Simulation by Qubitization](https://quantum-journal.org/papers/q-2019-07-12-163/).     Low et al. 2019. Sec 3.1, page 7 and 8 for high level overview and definitions. A     block encoding is called a standard form encoding there.
 - [The power of block-encoded matrix powers: improved regression techniques via faster Hamiltonian simulation](https://arxiv.org/abs/1804.01973).     Chakraborty et al. 2018. Definition 3 page 8.


In [ ]:
from qualtran.bloqs.block_encoding import LCUBlockEncoding

### Example Instances

In [ ]:
from qualtran.bloqs.chemistry.hubbard_model.qubitization import PrepareHubbard, SelectHubbard

# 3x3 hubbard model U/t = 4
dim = 3
select = SelectHubbard(x_dim=dim, y_dim=dim)
U = 4
t = 1
prepare = PrepareHubbard(x_dim=dim, y_dim=dim, t=t, u=U)
N = dim * dim * 2
qlambda = 2 * N * t + (N * U) // 2
lcu_block = LCUBlockEncoding(select=select, prepare=prepare, alpha=qlambda, epsilon=0.0)

In [ ]:
from qualtran.bloqs.block_encoding import BlackBoxPrepare, BlackBoxSelect
from qualtran.bloqs.chemistry.hubbard_model.qubitization import PrepareHubbard, SelectHubbard

# 3x3 hubbard model U/t = 4
dim = 3
select = SelectHubbard(x_dim=dim, y_dim=dim)
U = 4
t = 1
prepare = PrepareHubbard(x_dim=dim, y_dim=dim, t=t, u=U)
N = dim * dim * 2
qlambda = 2 * N * t + (N * U) // 2
black_box_lcu_block = LCUBlockEncoding(
    select=BlackBoxSelect(select), prepare=BlackBoxPrepare(prepare), alpha=qlambda, epsilon=0.0
)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([lcu_block, black_box_lcu_block],
           ['`lcu_block`', '`black_box_lcu_block`'])

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
lcu_block_g, lcu_block_sigma = lcu_block.call_graph(max_depth=1, generalizer=ignore_split_join)
show_call_graph(lcu_block_g)
show_counts_sigma(lcu_block_sigma)

## `LCUBlockEncodingZeroState`
LCU based block encoding using SELECT and PREPARE oracles.

Builds the standard block encoding from an LCU as
$$
    B[H] = \mathrm{PREPARE}^\dagger \cdot \mathrm{SELECT} \cdot \mathrm{PREPARE},
$$
where
$$
    \mathrm{PREPARE} |0\rangle_a = \sum_l \sqrt{\frac{w_l}{\alpha}} |l\rangle_a,
$$
and
$$
    \mathrm{SELECT} |l\rangle_a|\psi\rangle_s = |l\rangle_a U_l |\psi\rangle_s.
$$

The Hamiltonian can be extracted via

$$
    \langle G | B[H] | G \rangle = H / \alpha,
$$
where $|G\rangle_a = I_a |0\rangle_a$

The ancilla register is at least of size $\log L$.

In our implementations we typically split the ancilla registers into
selection registers (i.e.  the $l$ registers above) and junk registers which
are extra qubits needed by state preparation but not controlled upon during
SELECT.

#### Parameters
 - `alpha`: The normalization constant upper bounding the spectral norm of the Hamiltonian. Often called lambda.
 - `epsilon`: The precision to which the block encoding is performed. Currently this isn't used: see https://github.com/quantumlib/Qualtran/issues/985
 - `select`: The bloq implementing the `SelectOracle` interface.
 - `prepare`: The bloq implementing the `PrepareOracle` interface. 

#### Registers
 - `selection`: The combined selection register.
 - `junk`: Additional junk registers not prepared upon.
 - `system`: The combined system register. 

#### References
 - [Hamiltonian Simulation by Qubitization](https://quantum-journal.org/papers/q-2019-07-12-163/).     Low et al. 2019. Sec 3.1, page 7 and 8 for high level overview and definitions. A     block encoding is called a standard form encoding there.
 - [The power of block-encoded matrix powers: improved regression techniques via faster Hamiltonian simulation](https://arxiv.org/abs/1804.01973).     Chakraborty et al. 2018. Definition 3 page 8.


In [ ]:
from qualtran.bloqs.block_encoding import LCUBlockEncodingZeroState

### Example Instances

In [ ]:
from qualtran.bloqs.chemistry.hubbard_model.qubitization import PrepareHubbard, SelectHubbard

# 3x3 hubbard model U/t = 4
dim = 3
select = SelectHubbard(x_dim=dim, y_dim=dim)
U = 4
t = 1
prepare = PrepareHubbard(x_dim=dim, y_dim=dim, t=t, u=U)
N = dim * dim * 2
qlambda = 2 * N * t + (N * U) // 2
lcu_zero_state_block = LCUBlockEncodingZeroState(
    select=select, prepare=prepare, alpha=qlambda, epsilon=0.0
)

In [ ]:
from qualtran.bloqs.block_encoding import BlackBoxPrepare, BlackBoxSelect
from qualtran.bloqs.chemistry.hubbard_model.qubitization import PrepareHubbard, SelectHubbard

# 3x3 hubbard model U/t = 4
dim = 3
select = SelectHubbard(x_dim=dim, y_dim=dim)
U = 4
t = 1
prepare = PrepareHubbard(x_dim=dim, y_dim=dim, t=t, u=U)
N = dim * dim * 2
qlambda = 2 * N * t + (N * U) // 2
black_box_lcu_zero_state_block = LCUBlockEncodingZeroState(
    select=BlackBoxSelect(select), prepare=BlackBoxPrepare(prepare), alpha=qlambda, epsilon=0.0
)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([lcu_zero_state_block, black_box_lcu_zero_state_block],
           ['`lcu_zero_state_block`', '`black_box_lcu_zero_state_block`'])

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
lcu_zero_state_block_g, lcu_zero_state_block_sigma = lcu_zero_state_block.call_graph(max_depth=1, generalizer=ignore_split_join)
show_call_graph(lcu_zero_state_block_g)
show_counts_sigma(lcu_zero_state_block_sigma)